pandas-ylt
==========

Python modules for working in pandas with various loss data tables used for CAT modelling


### Basic usage
Read in from a csv file, assign the index

In [1]:
import pandas as pd
import numpy as np
import pandas_ylt

# Parameters
ifile = "tests/_data/example_pareto_poisson_yelt.csv"
index_cols = ['Year', 'EventID', 'DayOfYear']
loss_col = 'Loss'
n_years = 1e5

# Read the file as a pandas series
yelt = pd.read_csv(ifile, index_col=index_cols)[loss_col].sort_index()

# Display the file
yelt

Year   EventID  DayOfYear
1      33305    232          2540.574756
       45902    189          1313.996261
2      30597    54           1468.851587
4      10078    63           1677.475382
5      7014     9            1852.600528
                                ...     
99994  38943    115          1168.206535
99995  12281    126          2262.909615
99997  25978    55           1099.812510
       33186    120          3636.796884
99998  37515    49           2485.962160
Name: Loss, Length: 49812, dtype: float64

This is a Year Event Loss Table, and it can use the `.yel` series accessor. The only thing needed to be a loss series is an attribute `n_yrs`, which is set via the series `attrs`.

In [2]:
print(yelt.attrs)
print("AAL when n_yrs set in-line: ", yelt.yel.set_n_yrs(n_years).ls.aal)

yelt.attrs['n_yrs'] = n_years
print(yelt.attrs)
print("AAL when n_yrs already in the file: ", yelt.yel.aal)

print("Manual Calc:", yelt.sum() / n_years)


{}
AAL when n_yrs set in-line:  5563.82929844631
{'n_yrs': 100000.0}
AAL when n_yrs already in the file:  5563.82929844631
Manual Calc: 5563.82929844631


### Calculating a YLT
We can convert this series into a Year Loss Table, i.e. one loss for each year. The YLT uses the `.yl` series accessor.

In [3]:
ylt = yelt.yel.to_ylt()
print(f"AAL from ylt: {ylt.yl.aal:,.0f}")

ylt

AAL from ylt: 5,564


Year
1         3854.571017
2         1468.851587
4         1677.475382
5        34999.439551
6         1221.591205
             ...     
99992     1690.163154
99994    12014.849030
99995     2262.909615
99997     4736.609394
99998     2485.962160
Name: Loss, Length: 39246, dtype: float64

Standard deviation, taking account that missing years would be classed as zero.

In [4]:
# From a YLT we can calculate the std deviation. Arguments pass to pandas std
print(f"Std Dev from ylt: {ylt.yl.std():,.0f}")
print(f"Std Dev from ylt with 0 degrees of freedom: {ylt.yl.std(ddof=0):,.0f}")
print(f"Incorrect std Dev from pandas if we don't take account of zero loss years: {ylt.std():,.0f}")


Std Dev from ylt: 158,881
Std Dev from ylt with 0 degrees of freedom: 158,880
Incorrect std Dev from pandas if we don't take account of zero loss years: 253,376


### Calculating an AEP curve from the YLT

Through the accessor, we can calculate the exceedance probability of each loss. The below example shows how to append the output of that with the original losses into one table.

In [5]:
full_aep_curve = (
    ylt.to_frame()
    .assign(ExProb=ylt.yl.exprob())
    .assign(RP=ylt.yl.exprob().pow(-1))
).sort_values("ExProb")

number_format = {"Loss": "{:,.0f}", 'ExProb': '{:.5f}', 'RP': "{:,.0f}"}
display(full_aep_curve.head().style.format(number_format))


,Loss,ExProb,RP
Year,,,
22644,"36,281,334",0.00001,"100,000"
79142,"16,637,613",0.00002,"50,000"
54662,"12,056,465",0.00003,"33,333"
10432,"10,798,226",0.00004,"25,000"
51369,"9,848,174",0.00005,"20,000"


### Calculating an OEP curve from the YELT
To get an OEP curve, we first calculate the YLT for the maximum loss in each year

In [6]:
ylt_max = yelt.yel.to_ylt(is_occurrence=True)
ylt.to_frame().join(ylt_max.rename('MaxLoss'))

,Loss,MaxLoss
Year,,
1,3854.571017,2540.574756
2,1468.851587,1468.851587
4,1677.475382,1677.475382
5,34999.439551,31527.305894
6,1221.591205,1221.591205
...,...,...
99992,1690.163154,1690.163154
99994,12014.849030,10846.642495
99995,2262.909615,2262.909615


In [7]:
full_oep_curve = (
    ylt_max.to_frame()
    .assign(ExProb=ylt_max.yl.exprob())
    .assign(RP=ylt_max.yl.exprob().pow(-1))
).sort_values("ExProb")

display(full_oep_curve.head().style.format(number_format))

,Loss,ExProb,RP
Year,,,
22644,"36,281,334",0.00001,"100,000"
79142,"16,637,613",0.00002,"50,000"
54662,"12,056,465",0.00003,"33,333"
10432,"10,798,226",0.00004,"25,000"
51369,"9,848,174",0.00005,"20,000"


### Some nuances about the EP curve
When two losses are equal, the return period is equal. So you can't be guaranteed to know which return periods will be there.

This is illustrated below by the case where some losses are ceded to a layer

In [8]:
xs = 1e7
limit = 7e6
layer_ylt = (ylt - xs).clip(lower=0.0, upper=limit)
modified_ylt = ylt - layer_ylt

(modified_ylt
 .to_frame()
 .assign(ExProb=modified_ylt.yl.exprob())
 .assign(RP=modified_ylt.yl.exprob().pow(-1)) 
 ).sort_values('ExProb').head(5)

,Loss,ExProb,RP
Year,,,
22644,2.928133e+07,0.00001,100000.0
10432,1.000000e+07,0.00004,25000.0
79142,1.000000e+07,0.00004,25000.0
54662,1.000000e+07,0.00004,25000.0
51369,9.848174e+06,0.00005,20000.0


In some use-cases, we may just want the EP curve without duplicates, but then we'd lose the connection back to the source years so the dataframe is no longer a YLT

In [9]:
print(f"Length of the YLT: {len(modified_ylt):,d}")
ep_curve = modified_ylt.yl.to_ep_curve()
print(f"Length of the EP curve: {len(ep_curve):,d}")

ep_curve.head(5).style.format(number_format)

Length of the YLT: 39,246
Length of the EP curve: 39,245


,Loss,ExProb
Order,,
0,"29,281,334",0.00001
1,"10,000,000",0.00004
2,"9,848,174",0.00005
3,"8,785,562",0.00006
4,"7,380,617",0.00007


In other use cases, we may want to know the loss for specified exceedance probabilities

In [10]:
exprobs = np.arange(1, 7) / n_years
print(exprobs)

losses_exceeded = modified_ylt.yl.loss_at_exprobs(exprobs)
print(losses_exceeded)

pd.DataFrame({'ExProb': exprobs,
              'Loss': losses_exceeded}).style.format(number_format)

[1.e-05 2.e-05 3.e-05 4.e-05 5.e-05 6.e-05]
[29281334.36227607 10000000.         10000000.         10000000.
  9848174.09097693  8785562.23125718]


,ExProb,Loss
0,0.00001,"29,281,334"
1,0.00002,"10,000,000"
2,0.00003,"10,000,000"
3,0.00004,"10,000,000"
4,0.00005,"9,848,174"
5,0.00006,"8,785,562"


Note that when you provide invalid probabilities, it will return NaN

In [11]:
exprobs = [1 / r for r in [1000000, 100000, 10000, 1000, 100, 10, 1.0, 0.1]]
modified_ylt.yl.loss_at_exprobs(exprobs)

array([           nan, 2.92813344e+07, 6.53395618e+06, 4.94220890e+05,
       5.16105644e+04, 5.53896022e+03, 0.00000000e+00,            nan])

We can also look at the case where we want to know the probability of exceeding specific losses

In [12]:
losses = [30e6, 20e6, 10e6, 30e5, 20e5, 10e5]
print(losses)

exprobs = modified_ylt.yl.loss_exprobs(losses)
print(exprobs)

pd.DataFrame({'Loss': losses,'ExProb': exprobs}).style.format(number_format)

[30000000.0, 20000000.0, 10000000.0, 3000000.0, 2000000.0, 1000000.0]
[0.0e+00 1.0e-05 4.0e-05 2.0e-04 2.6e-04 5.1e-04]


,Loss,ExProb
0,"30,000,000",0.00000
1,"20,000,000",0.00001
2,"10,000,000",0.00004
3,"3,000,000",0.00020
4,"2,000,000",0.00026
5,"1,000,000",0.00051


#### Note: Definition of exceedance probability
The exceedance probability is commonly defined as P(X>=x). This makes it slightly different to a survivor function, which would be 1 - P(X<=x) = P(X>x)

In [13]:
(modified_ylt
 .to_frame()
 .assign(ExProb=modified_ylt.yl.exprob(method='min'))
  ).sort_values('ExProb').head(5)

,Loss,ExProb
Year,,
22644,2.928133e+07,0.00001
10432,1.000000e+07,0.00002
79142,1.000000e+07,0.00002
54662,1.000000e+07,0.00002
51369,9.848174e+06,0.00005


In [14]:
exprobs = np.arange(1, 7) / n_years
print(exprobs)

losses_exceeded = modified_ylt.yl.loss_at_exprobs(exprobs, method='min')
print(losses_exceeded)

pd.DataFrame({'ExProb': exprobs,
              'Loss': losses_exceeded}).style.format(number_format)

[1.e-05 2.e-05 3.e-05 4.e-05 5.e-05 6.e-05]
[29281334.36227607 10000000.          9848174.09097693  9848174.09097693
  9848174.09097693  8785562.23125718]


,ExProb,Loss
0,0.00001,"29,281,334"
1,0.00002,"10,000,000"
2,0.00003,"9,848,174"
3,0.00004,"9,848,174"
4,0.00005,"9,848,174"
5,0.00006,"8,785,562"


This would be the appropriate method if we wanted to get percentiles for a cumulative distribution.

In [15]:
cprobs = np.linspace(0, 1, 11)
cprobs

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

### Easy way to get the EP curve

#### From a YLT
There is also a function to get the loss at specified exceedance probabilities, but taking return periods as arguments and returning as a series.

In [16]:
return_periods = [100000, 10000, 1000, 500, 200, 100, 50, 20, 10]
ylt.yl.to_rp_summary(return_periods).to_frame().style.format(number_format)

,Loss
ReturnPeriod,
100000,"36,281,334"
10000,"6,533,956"
1000,"494,221"
500,"276,674"
200,"104,018"
100,"51,611"
50,"25,875"
20,"10,961"
10,"5,539"


### From a YELT

In [17]:
(yelt.yel.to_rp_summary(return_periods, is_ep=True, is_occurrence=False)
 .to_frame().style.format(number_format))

,Loss
ReturnPeriod,
100000,"36,281,334"
10000,"6,533,956"
1000,"494,221"
500,"276,674"
200,"104,018"
100,"51,611"
50,"25,875"
20,"10,961"
10,"5,539"


In [18]:
(yelt.yel.to_rp_summaries(return_periods, is_ef=False)
  .rename('Loss').to_frame().style.format(number_format)
 )

### Exceedance Frequency from a YELT
Where events are concerned, we may want to look at the exceedance frequency or rate

In [19]:
full_ef_curve = (
    yelt.to_frame()
    .assign(ExFreq=yelt.yel.exfreq())
    .assign(RP=yelt.yel.exfreq().pow(-1))
).sort_values("ExFreq")

display(full_ef_curve.head().style.format(number_format))
display(full_ef_curve.tail().style.format(number_format))


,,,Loss,ExFreq,RP
Year,EventID,DayOfYear,,,
22644,43737,354,"36,281,334",0.000010,"100,000"
79142,11708,90,"16,637,613",0.000020,"50,000"
54662,3586,186,"12,056,465",0.000030,"33,333"
10432,28920,362,"10,798,226",0.000040,"25,000"
51369,29355,224,"9,848,174",0.000050,"20,000"


,,,Loss,ExFreq,RP
Year,EventID,DayOfYear,,,
5440,12820,85,"1,000",0.498080,2
50809,38489,5,"1,000",0.498090,2
32213,17253,232,"1,000",0.498100,2
37523,37685,105,"1,000",0.498110,2
35617,26476,233,"1,000",0.498120,2


In [20]:
print(f"Length of the YELT: {len(yelt):,d}")
ef_curve = yelt.yel.to_ef_curve()
print(f"Length of the EF curve: {len(ef_curve):,d}")

ef_curve.head(5).style.format(number_format)

Length of the YELT: 49,812
Length of the EF curve: 49,812


,Loss,ExFreq
Order,,
0,"36,281,334",0.000010
1,"16,637,613",0.000020
2,"12,056,465",0.000030
3,"10,798,226",0.000040
4,"9,848,174",0.000050


## Alternative YELT format
In this case there is a separate index level for the Model ID, which categorises groups of events. There are also two loss perspectives stored in separate columns of a dataframe. 

In [21]:
yelts = pd.read_csv("tests/_data/example_two_models_grossnet.csv")
yelts = yelts.set_index(["Year", "ModelID", "EventID", "DayOfYear"])
yelts.attrs["n_yrs"] = 1e5
display(yelts)

# Convert to a series
yelts = pd.concat(
    [yelts["GrossLoss"].rename("Loss"), yelts["NetLoss"].rename("Loss")],
    keys=["Gross", "Net"],
    names=["Persp"],
)
yelts = yelts.loc[yelts > 0.0]
yelts

GrossLoss  NetLoss
Year   ModelID EventID DayOfYear                      
1      Model1  45902   189        1313.996261      0.0
               33305   232        2540.574756      0.0
2      Model1  30597   54         1468.851587      0.0
4      Model1  10078   63         1677.475382      0.0
5      Model1  7014    9          1852.600528      0.0
...                                       ...      ...
99995  Model2  19599   54         1024.742827      0.0
               40252   362        2469.358250      0.0
99999  Model2  72370   2          3170.787193      0.0
               83257   282        3533.693832      0.0
100000 Model2  48460   315        1016.256966      0.0

[149727 rows x 2 columns]

Persp  Year   ModelID  EventID  DayOfYear
Gross  1      Model1   45902    189           1313.996261
                       33305    232           2540.574756
       2      Model1   30597    54            1468.851587
       4      Model1   10078    63            1677.475382
       5      Model1   7014     9             1852.600528
                                                 ...     
Net    99184  Model2   63158    114          10132.664034
       99321  Model2   72176    214           3165.905923
       99419  Model2   46940    221          40257.608715
       99672  Model2   10385    239          37546.432166
       99778  Model2   85895    43            2709.836547
Name: Loss, Length: 153290, dtype: float64

### EP curves on different partitions of the YELT

In [22]:
# AEP on the net loss
is_subset = yelts.index.get_level_values('Persp') == 'Net'

(yelts.loc[is_subset].yel.to_rp_summary(return_periods)
 .to_frame().style.format(number_format)
)

,Loss
ReturnPeriod,
100000,"188,905"
10000,"120,000"
1000,"120,000"
500,"120,000"
200,"99,561"
100,"42,745"
50,"12,830"
20,0
10,0


In [23]:
split_by = ['Persp', 'ModelID']
return_periods = [1000, 100, 10]

ep_curves = {grp: df.yel.to_rp_summaries(return_periods, is_ef=False)
             for grp, df in yelts.groupby(split_by)}

ep_curves = pd.concat(ep_curves.values(), keys=ep_curves.keys(), names=split_by)

display(ep_curves.to_frame().style.format("{:,.0f}"))


We can append the summary stats to the same table

In [24]:
summary_stats = {grp: df.yel.to_summary_stats_series()
             for grp, df in yelts.groupby(split_by)}

summary_stats = pd.concat(summary_stats.values(), keys=summary_stats.keys(),
                          names=split_by)

display(summary_stats.to_frame().style.format("{:,.0f}"))

display(pd.concat([ep_curves.reset_index(), 
                   summary_stats.reset_index().assign(ReturnPeriod=0)])
                   .sort_values(split_by)
                   .set_index(ep_curves.index.names)
                   .style.format(number_format))